# Exercise 8: Cues, Wanting, and the Reward System
## A quick look at the MID task

In the **Monetary Incentive Delay (MID)** task:

1. Participants see a cue that signals a possible reward, neutral outcome, or loss.  
2. They respond quickly to a target to try to earn money (or avoid losing it).  
3. We focus on the **cue period** to study *wanting* (motivational salience) rather than *liking*.  

> **See “Guide to Exercise 8” slides** for a step‑by‑step demo of the MID timing, trial types, and expected brain responses.  
> **Reminder:** PSC is BOLD percent‑signal change (blood‑flow proxy), not dopamine.

---

## What you will do in this notebook (and why)

### 1. **QC Anatomy**  
You will load brain images and verify the alignment of your region of interest (ROI)—the nucleus accumbens (NAcc).  
**Why?**  
✅ To make sure the mask correctly covers the NAcc (more on the mask below). If the mask is off, any measurements from it would not represent true NAcc activity.

#### Exploring the NAcc in the ABCD Data Dictionary

For a deeper understanding of the nucleus accumbens (NAcc), look up the following metrics in the ABCD data dictionary and visualize them using the Brain Atlas Visualizer:
https://abcd.deapscience.com/#/my-datasets/create-dataset

1. **mr_y_smri__vol__aseg__ab__lh_sum (and _rh_sum)**
   - **What it measures:** Total volume (mm³) of the left/right nucleus accumbens from FreeSurfer’s subcortical segmentation.
   - **Why it’s useful:** This is the classic structural metric—literally the size of the NAcc. It’s stable, easy to interpret, and visually clear when you explore the region in the ABCD Brain Atlas Visualizer.

2. **mr_y_smri__t1__aseg__ab__lh_mean (and _rh_mean)**
   - **What it measures:** Mean T1-weighted intensity within the NAcc (left/right).
   - **Why it’s useful:** This reflects tissue contrast and integrity. It isn’t a “size” measure but a signal measure related to microstructural properties (e.g., myelination, iron content, water density). It provides a complementary way of looking at NAcc structure beyond just volume.
---

### 2. **Extract NAcc Time-Series**  
You will extract the BOLD signal (functional MRI activation) from the NAcc across time. “Blood‑Oxygen‑Level‑Dependent (BOLD) signal”: this is a measure of blood flow to a brain region, which we use as a proxy for neural activity. 

**Why?**  
✅ To isolate the activity from our key brain region, so we can later connect its behavior to cue periods and trial events.

---

### 3. **Build a Trial Table**  
You will build a tidy trial‑level table for **cue trials only**, with reward vs neutral labels.  
**Why?**  
✅ A clean table makes it easy to compare conditions and plot results consistently across ROIs.  
🧠 The **incentive‑salience hypothesis** proposes that dopamine transforms neutral cues into “wanted” signals, giving them motivational power to drive behavior.

---

### 4. **Visualizations**  
You will generate several figures:  
- Cue‑locked time series  
- Box plots of cue‑window means  
- Coupling scatter and a tiny summary table  

**Why?**  
✅ Different plots let you see the data from multiple perspectives: central tendency, spread, and brain location.  
✅ Visual summaries make it easier to compare ROIs and conditions.

---

Let’s get started! 🚀

In [ ]:
# ════════════════════════════════════════════════════════════════════
# Import Libraries & Configure Environment
# ════════════════════════════════════════════════════════════════════

# Core libraries
import os
from pathlib import Path
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# fMRI analysis
from nilearn import image, masking
from nilearn.image import resample_to_img
from nilearn.masking import apply_mask

# Interactive widgets
from ipywidgets import interact
from IPython.display import Image, display

# Network utilities (for data download)
import urllib.request
import gzip
import shutil

# ────────────────────────────────────────────────────────────────────
# Configuration
# ────────────────────────────────────────────────────────────────────

# Display plots inline
%matplotlib inline

# Set visualization style
sns.set_theme(context="notebook", style="whitegrid", palette="muted", font_scale=1.2)

# Create output folder for figures
Path("figs").mkdir(exist_ok=True)

print("✅ Environment configured successfully")

---
## Data Source, Citation & Folder Layout

**🚨 TEAMWORK NOTE (read carefully!)**  
You can work with any of the four available subjects:
- **Subject 06:** set `SUBJECT = "sub-06"`  
- **Subject 09:** set `SUBJECT = "sub-09"`  
- **Subject 11:** set `SUBJECT = "sub-11"`  
- **Subject 14:** set `SUBJECT = "sub-14"`  

In this exercise we use **real fMRI data** from the Adolescent Health and Development in Context (AHDC) study, publicly available on OpenNeuro:

> Baldwin M. Way, Christopher R. Browning, Dylan D. Wagner, Jodi L. Ford, Bethany Boettner & Ping Bai (2025).  
> _Structural and functional MRI dataset from the Adolescent Health and Development in Context (AHDC) study in Columbus, Ohio._  
> OpenNeuro [Dataset] doi:10.18112/openneuro.ds005901.v1.0.0  
> https://github.com/OpenNeuroDatasets/ds005901

This dataset was collected as part of a **longitudinal neuroimaging study of adolescent health and development**. Participants completed surveys, smartphone-based ecological momentary assessments, and MRI scans across multiple waves. One aim was to understand how **community exposures and reward processes** shape brain function and substance use risk.

We'll focus on the **Monetary Incentive Delay (MID)** task, which measures reward anticipation and cue-related responses. You can choose to analyze any of the four subjects' data from the first MID run.

> **Before you begin, set `SUBJECT = "sub-06"`, `"sub-09"`, `"sub-11"`, or `"sub-14"`** in the code cell below.

Each subject's folder contains:  
- a 4D BOLD fMRI volume:  
  `sub-06_task-mid_run-01_bold.nii.gz`  
- its trial timing file:  
  `sub-06_task-mid_run-01_events.tsv`  
- (you'll also need the bilateral NAcc ROI mask: `nacc_bilateral_mask.nii`, placed in the `data/` folder)

**Expected directory structure (this repo):**

```
Exercise8/
├── Exercise8.ipynb ← This notebook
├── figs/ ← Saved figures will go here
└── data/
    ├── nacc_bilateral_mask.nii
    ├── qc_nacc_roi_alignment.png ← Static QC image (if provided)
```


In [ ]:
# ──────────────────────────────────

# 0. Data folder constants

# ──────────────────────────────────

# Data will be stored in the 'data' subfolder relative to this notebook
# This works whether the notebook is opened directly or from the repo root
DATA_ROOT = Path("data")

print(f"📁 Data root: {DATA_ROOT.resolve()}")


## Select Your Subject

Choose which subject's data you'd like to analyze. Available subjects: sub-06, sub-09, sub-11, sub-14.

> **Set `SUBJECT = "sub-06"`, `"sub-09"`, `"sub-11"`, or `"sub-14"`** in the cell below.

In [ ]:
# Change this to "sub-06", "sub-09", "sub-11", or "sub-14" before you start!

SUBJECT = "sub-06"

# Map classroom-friendly IDs to OpenNeuro dataset IDs
SUBJECT_MAP = {
    "sub-06": "sub-s006",
    "sub-09": "sub-s009",
    "sub-11": "sub-s011",
    "sub-14": "sub-s014",
}

if not isinstance(SUBJECT, str):
    raise ValueError("SUBJECT must be a string.")

subject_norm = SUBJECT.strip().lower()

# Accept common variants and normalize:
# - sub-06 / sub-09 / sub-11 / sub-14 (preferred classroom labels)
# - sub-006 / sub-009 / sub-011 / sub-014
# - sub-s006 / sub-s009 / sub-s011 / sub-s014 (direct dataset IDs)
if subject_norm in SUBJECT_MAP:
    DATASET_SUBJECT = SUBJECT_MAP[subject_norm]
elif subject_norm.startswith("sub-") and subject_norm[4:].isdigit():
    n = int(subject_norm[4:])
    candidate = f"sub-{n:02d}"
    if candidate in SUBJECT_MAP:
        DATASET_SUBJECT = SUBJECT_MAP[candidate]
    else:
        raise ValueError(
            "Only subjects 06, 09, 11, and 14 are available in this exercise."
        )
elif subject_norm.startswith("sub-s") and subject_norm[5:].isdigit():
    n = int(subject_norm[5:])
    candidate = f"sub-{n:02d}"
    if candidate in SUBJECT_MAP:
        DATASET_SUBJECT = subject_norm
    else:
        raise ValueError(
            "Only subjects 06, 09, 11, and 14 are available in this exercise."
        )
else:
    raise ValueError(
        "SUBJECT must be one of 'sub-06', 'sub-09', 'sub-11', 'sub-14' (or variants like 'sub-006' / 'sub-s006')."
    )

# Pre-defined file paths (dataset uses sub-sXXX folders/files)
FUNC_DIR = DATA_ROOT / DATASET_SUBJECT / "func"
BOLD_PATH = FUNC_DIR / f"{DATASET_SUBJECT}_task-mid_run-01_bold.nii.gz"
EVENT_PATH = FUNC_DIR / f"{DATASET_SUBJECT}_task-mid_run-01_events.tsv"
NACC_MASK_PATH = DATA_ROOT / "nacc_bilateral_mask.nii"

FIG_DIR = Path("figs")
FIG_DIR.mkdir(exist_ok=True)

print("Selected subject (input):", SUBJECT)
print("Selected subject (dataset ID):", DATASET_SUBJECT)
print("  BOLD (compressed) →", BOLD_PATH)
print("  EVENTS →", EVENT_PATH)
print("  NAcc MASK →", NACC_MASK_PATH)
print("  FIGS →", FIG_DIR.resolve())

## Create a VTA Sphere Mask (in BOLD space)

We'll build a small spherical ROI for the **ventral tegmental area (VTA)** directly inside the notebook. Run the download cell first so the BOLD file exists.

- **VTA center (MNI):** $(-4, -15, -9)$  
- **Radius:** 5 mm (small enough to target midbrain while still capturing a few voxels)

This mask is created in the same voxel space as your BOLD image, then saved to `data/vta_sphere_mask.nii.gz`.

In [ ]:
# ════════════════════════════════════════════════════════════════════
# Robust data download (OpenNeuro snapshot URL + direct S3 fallback; GitHub fallback for TSV only)
# ════════════════════════════════════════════════════════════════════

from pathlib import Path
import os
import shutil
import subprocess
import urllib.request
from urllib.error import HTTPError, URLError

DATASET  = "ds005901"
SNAPSHOT = "1.0.0"  # used by OpenNeuro snapshot URLs

# Ensure output folders exist early
FUNC_DIR.mkdir(parents=True, exist_ok=True)

# --------------------------------------------------------------------
# Helpers
# --------------------------------------------------------------------
def is_gzip_file(path: Path) -> bool:
    """Return True if file exists and has gzip magic bytes."""
    try:
        with open(path, "rb") as f:
            return f.read(2) == b"\x1f\x8b"
    except OSError:
        return False


def download_one(url: str, dest: Path) -> bool:
    """Download URL to dest, streaming in chunks. Returns True on success."""
    try:
        dest.parent.mkdir(parents=True, exist_ok=True)
        with urllib.request.urlopen(url) as r, open(dest, "wb") as f:
            shutil.copyfileobj(r, f, length=1024 * 1024)
        return True
    except (HTTPError, URLError) as e:
        print(f"   ⚠️  failed: {e}")
        return False
    except Exception as e:
        print(f"   ⚠️  failed: {type(e).__name__}: {e}")
        return False


def download_with_fallback(urls: list[str], dest: Path) -> Path | None:
    """Try URLs in order; returns dest if downloaded, else None."""
    for i, url in enumerate(urls, start=1):
        print(f"⬇️  [{i}/{len(urls)}] {dest.name} ← {url}")
        if download_one(url, dest):
            print("   …downloaded")
            return dest
    return None


def ensure_openneuro_py() -> None:
    """Install openneuro-py once per session if needed."""
    try:
        subprocess.run(
            ["openneuro-py", "--help"],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL,
            check=True,
        )
    except Exception:
        subprocess.run(["python3", "-m", "pip", "install", "-q", "openneuro-py"], check=True)


def openneuro_py_download(relpath: str, target_dir: str) -> None:
    """
    relpath example: 'sub-s006/func/sub-s006_task-mid_run-01_bold.nii.gz'
    Downloads into target_dir/..., preserving the relpath structure.
    """
    ensure_openneuro_py()
    subprocess.run(
        [
            "openneuro-py",
            "download",
            f"--dataset={DATASET}",
            f"--target-dir={target_dir}",
            f"--include={relpath}",
        ],
        check=True,
    )

# --------------------------------------------------------------------
# Build paths + URLs
# --------------------------------------------------------------------
events_fname = EVENT_PATH.name
bold_fname   = BOLD_PATH.name

evt_path     = EVENT_PATH
bold_path_gz = BOLD_PATH

# OpenNeuro "snapshot" URLs (sometimes return 502 during outages)
openneuro_evt = (
    f"https://openneuro.org/crn/datasets/{DATASET}/snapshots/{SNAPSHOT}/files/"
    f"{DATASET_SUBJECT}:func:{events_fname}?download=1"
)
openneuro_bold = (
    f"https://openneuro.org/crn/datasets/{DATASET}/snapshots/{SNAPSHOT}/files/"
    f"{DATASET_SUBJECT}:func:{bold_fname}?download=1"
)

# Direct S3 URLs (often work even if OpenNeuro web/API is down)
s3_evt  = f"https://s3.amazonaws.com/openneuro.org/{DATASET}/{DATASET_SUBJECT}/func/{events_fname}"
s3_bold = f"https://s3.amazonaws.com/openneuro.org/{DATASET}/{DATASET_SUBJECT}/func/{bold_fname}"

# GitHub raw: OK for TSV; DO NOT use GitHub raw for .nii.gz (git-annex pointer)
github_evt = f"https://github.com/OpenNeuroDatasets/{DATASET}/raw/main/{DATASET_SUBJECT}/func/{events_fname}"

events_urls = [openneuro_evt, s3_evt, github_evt]
bold_urls   = [openneuro_bold, s3_bold]

print(f"🔍 Checking data files for {SUBJECT} (dataset: {DATASET_SUBJECT})…")

# EVENTS (idempotent)
if evt_path.exists() and evt_path.stat().st_size > 0:
    print(f"✅ {events_fname} already exists, skipping download.")
else:
    if download_with_fallback(events_urls, evt_path) is None:
        print("❌ Events download failed.")

# BOLD (idempotent + gzip check)
if bold_path_gz.exists() and is_gzip_file(bold_path_gz):
    print(f"✅ {bold_fname} already exists and is gzip, skipping download.")
else:
    bold_gz = download_with_fallback(bold_urls, bold_path_gz)

    # If somehow a non-gzip file landed at the destination, delete it
    if bold_gz and not is_gzip_file(bold_gz):
        print("🧪 The downloaded BOLD file is not gzip. Deleting it.")
        try:
            os.remove(bold_gz)
        except OSError:
            pass
        bold_gz = None

    # If direct URL routes failed, try openneuro-py (may also fail during outages)
    if bold_gz is None:
        try:
            print("🔁 Trying OpenNeuro via openneuro-py…")
            rel = f"{DATASET_SUBJECT}/func/{bold_fname}"
            openneuro_py_download(rel, target_dir=str(DATA_ROOT))
        except Exception as e:
            print(f"❌ openneuro-py download failed: {e}")

print("\n======================================================================")
print("📍 File paths:")
print(f"  BOLD:   {BOLD_PATH} (exists: {BOLD_PATH.exists()})")
print(f"  EVENTS: {EVENT_PATH} (exists: {EVENT_PATH.exists()})")
print(f"  NAcc MASK: {NACC_MASK_PATH} (exists: {NACC_MASK_PATH.exists()})")

assert BOLD_PATH.exists() and is_gzip_file(BOLD_PATH), "Missing or invalid BOLD .nii.gz."
assert EVENT_PATH.exists(), "Missing events.tsv."
assert NACC_MASK_PATH.exists(), "Missing NAcc mask (data/nacc_bilateral_mask.nii)."


In [ ]:
import nibabel as nib

VTA_MNI = (-4, -15, -9)
VTA_RADIUS_MM = 5

if not BOLD_PATH.exists():
    raise FileNotFoundError("BOLD file not found. Run the download cell above first.")

bold_img = nib.load(str(BOLD_PATH))
affine = bold_img.affine
shape = bold_img.shape[:3]

# Build a voxel-wise distance map in MNI space
ijk = np.indices(shape).reshape(3, -1).T
xyz = nib.affines.apply_affine(affine, ijk)
center = np.array(VTA_MNI, dtype=float)
dist_mm = np.linalg.norm(xyz - center, axis=1)

mask_data = (dist_mm <= VTA_RADIUS_MM).reshape(shape).astype("uint8")

VTA_MASK_PATH = DATA_ROOT / "vta_sphere_mask.nii.gz"
mask_img = nib.Nifti1Image(mask_data, affine)
nib.save(mask_img, str(VTA_MASK_PATH))

print(f"✅ VTA mask saved: {VTA_MASK_PATH}")
print(f"   Center (MNI): {VTA_MNI}, radius: {VTA_RADIUS_MM} mm")

## 1. QC Anatomy & File Sanity Checks

We will generate two QC overlays and save them to `figs/`:
- `0_qc_nacc.png`: mean functional image + NAcc mask
- `0_qc_vta.png`: mean functional image + VTA sphere mask

Check that both ROIs sit in the expected anatomical locations before proceeding.

In [ ]:
# ════════════════════════════════════════════════════════════════════
# QC: file checks + generate NAcc and VTA ROI overlays on mean functional
# ════════════════════════════════════════════════════════════════════

import numpy as np
import nibabel as nib
from nilearn import image, plotting
from PIL import Image as PILImage
from IPython.display import display

print(f"📊 Analyzing: {SUBJECT}")
print("=" * 60)
print(f"  BOLD   exists? {BOLD_PATH.exists()}  -> {BOLD_PATH}")
print(f"  EVENTS exists? {EVENT_PATH.exists()} -> {EVENT_PATH}")
print(f"  NAcc  exists? {NACC_MASK_PATH.exists()}  -> {NACC_MASK_PATH}")
print(f"  VTA   exists? {VTA_MASK_PATH.exists()}  -> {VTA_MASK_PATH}")

# Stop early if anything required is missing
missing_files = []
if not BOLD_PATH.exists():
    missing_files.append(("BOLD", BOLD_PATH))
if not EVENT_PATH.exists():
    missing_files.append(("EVENTS", EVENT_PATH))
if not NACC_MASK_PATH.exists():
    missing_files.append(("NAcc_MASK", NACC_MASK_PATH))
if not VTA_MASK_PATH.exists():
    missing_files.append(("VTA_MASK", VTA_MASK_PATH))

if missing_files:
    print("\n❌ Missing required files:")
    for label, path in missing_files:
        print(f"  - {label}: {path}")

    print("\nFixes:")
    print("  - Re-run the download cell above to fetch BOLD and EVENTS.")
    print("  - Confirm the instructor-provided NAcc mask is present in data/.")
    print("  - Re-run the VTA mask cell to regenerate the VTA sphere.")
    raise FileNotFoundError("Required files missing; cannot generate QC figures.")

print("\n✅ All required files found. Generating QC overlay figures...")

# Helper: compute mask center-of-mass (mm) for sensible cut coordinates
def mask_center_mm(mask_path):
    m = nib.load(str(mask_path))
    data = m.get_fdata() > 0
    ijk = np.argwhere(data)
    if ijk.size == 0:
        return (0.0, 0.0, 0.0)
    com_ijk = ijk.mean(axis=0)
    com_xyz = nib.affines.apply_affine(m.affine, com_ijk)
    return (float(com_xyz[0]), float(com_xyz[1]), float(com_xyz[2]))

# Helper: parse task/run from filename for a clearer title
def parse_task_run(fname):
    task = None
    run = None
    for part in fname.split("_"):
        if part.startswith("task-"):
            task = part.replace("task-", "")
        if part.startswith("run-"):
            run = part.replace("run-", "")
    return task, run

task, run = parse_task_run(BOLD_PATH.name)
task_str = "MID task" if task == "mid" else (f"task-{task}" if task else "task")
run_str = f"run-{run}" if run else "run"

cut_coords_nacc = mask_center_mm(NACC_MASK_PATH)
cut_coords_vta = mask_center_mm(VTA_MASK_PATH)

qc_nacc_path = FIG_DIR / "0_qc_nacc.png"
qc_vta_path = FIG_DIR / "0_qc_vta.png"

# Always generate (overwrite) so the QC images are real, not placeholders
mean_func = image.mean_img(str(BOLD_PATH))

# QC Figure 1: NAcc overlay
disp_nacc = plotting.plot_roi(
    str(NACC_MASK_PATH),
    bg_img=mean_func,
    title=f"QC: NAcc ROI alignment (mean fMRI, {SUBJECT}, {task_str}, {run_str})",
    display_mode="ortho",
    cut_coords=cut_coords_nacc,
    draw_cross=True,
)
disp_nacc.savefig(str(qc_nacc_path), dpi=250)
disp_nacc.close()

# QC Figure 2: VTA overlay
disp_vta = plotting.plot_roi(
    str(VTA_MASK_PATH),
    bg_img=mean_func,
    title=f"QC: VTA sphere alignment (mean fMRI, {SUBJECT}, {task_str}, {run_str})",
    display_mode="ortho",
    cut_coords=cut_coords_vta,
    draw_cross=True,
)
disp_vta.savefig(str(qc_vta_path), dpi=250)
disp_vta.close()

print(f"\n📍 QC figures saved to:")
print(f"  - {qc_nacc_path}")
print(f"  - {qc_vta_path}")
print(f"   NAcc cut coords (mm): x={cut_coords_nacc[0]:.1f}, y={cut_coords_nacc[1]:.1f}, z={cut_coords_nacc[2]:.1f}")
print(f"   VTA  cut coords (mm): x={cut_coords_vta[0]:.1f}, y={cut_coords_vta[1]:.1f}, z={cut_coords_vta[2]:.1f}")

display(PILImage.open(qc_nacc_path))
display(PILImage.open(qc_vta_path))


### QC Checklist

- BOLD, EVENTS, NAcc mask, and VTA mask files exist.
- NAcc overlay sits in ventral striatum.
- VTA sphere sits in the midbrain; if it looks off-target, adjust `VTA_MNI` or `VTA_RADIUS_MM` and re-run the VTA cell.

## 2. Extract NAcc Time-Series

Before we dive in, let's simplify **TR** (Repetition Time):

> **TR = the interval between "snapshots."**  
> Every TR seconds (e.g. 2 s) the scanner takes one full 3D "photo" of your brain. When we want the BOLD signal right after an event, we convert the event time (in seconds) into which snapshot number (TR index) to pull.

In the cell below we use a helper function `extract_psc()` that:

1. **Loads** the 4D BOLD time-series image
2. **Resamples the ROI mask** to match the BOLD data's voxel grid and coordinate system  
   🧠 This ensures each mask voxel correctly overlaps with corresponding BOLD voxels
3. **Applies the mask** to extract BOLD values from NAcc voxels across all time points  
   🎯 Isolates the signal over time from just the NAcc—ignoring everything else in the brain
4. **Averages across voxels** at each TR to produce a single NAcc time-series  
5. **Converts to percent-signal-change (PSC)** for standardized, interpretable units

The function returns both:
- `nacc_psc`: percent-signal-change time-series (for analysis)
- `nacc_ts`: raw signal time-series (for reference)

In [ ]:
# ════════════════════════════════════════════════════════════════════
# 2. Extract ROI time-series (PSC)
# ════════════════════════════════════════════════════════════════════

def extract_psc(bold_path, mask_path, roi_name):
    """
    Extract percent-signal-change time-series from an ROI.

    Returns a dict with:
      - roi_name
      - raw_ts
      - psc_ts
      - baseline_raw
    """
    bold_img = image.load_img(str(bold_path))

    # Resample mask to BOLD space (nearest-neighbor keeps it binary)
    mask_res = resample_to_img(
        source_img=str(mask_path),
        target_img=bold_img,
        interpolation="nearest"
    )

    # apply_mask returns array of shape (n_volumes, n_voxels)
    roi_vox_ts = apply_mask(bold_img, mask_res)
    raw_ts = roi_vox_ts.mean(axis=1)

    baseline_raw = float(raw_ts.mean())
    psc_ts = 100 * (raw_ts - baseline_raw) / baseline_raw

    print(f"{roi_name}: {len(psc_ts)} TRs, baseline = {baseline_raw:.1f}")

    return {
        "roi_name": roi_name,
        "raw_ts": raw_ts,
        "psc_ts": psc_ts,
        "baseline_raw": baseline_raw,
    }

# 2a) Extract NAcc and VTA time-series
nacc = extract_psc(BOLD_PATH, NACC_MASK_PATH, "NAcc")
vta = extract_psc(BOLD_PATH, VTA_MASK_PATH, "VTA")

# 2b) Quick peek: show first k rows of the NAcc PSC time-series in a table
nacc_psc = nacc["psc_ts"]
df_nacc = pd.DataFrame(nacc_psc, columns=["psc_nacc"])

# interactive slider to choose how many rows to display
interact(
    lambda k: df_nacc.head(k),
    k=(1, min(20, len(df_nacc)), 1)  # slider from 1 up to 20 (or total TRs if fewer)
)

### What just happened?

- **Raw → PSC:** We calculated the average ROI signal across the entire run (`baseline_raw`), then converted each TR’s mean activation into **percent-signal-change** (`psc_ts`). Now “0 %” means no change from baseline, and “+1 %” means a 1 % increase.
- **What is PSC?** **Percent-Signal-Change (PSC)** expresses each TR’s BOLD signal as a percentage change from its baseline, effectively measuring the percent change in blood flow (and thus neural activity) within your ROI (Brain Region of Interest).
- **Interactive preview:** The slider lets you inspect the first k rows of the NAcc PSC time-series. Try moving it between 1 and 20 (or more) to see how NAcc activation evolves over those snapshots.
- **Why PSC?** Converting to percent-signal-change puts all activation values on an intuitive, comparable scale—standard practice in fMRI—so your subsequent plots speak in “percent change” rather than arbitrary intensities.


## 3. Build a Trial Table (Cue Only)

In this step we:

- **Load** the MID events file and read each trial's onset time (in seconds) and `trial_type`.
- **Extract** TR from the BOLD header and print it.
- **Keep only cue trials** (trial_type starts with `Cue`, case-insensitive).
- **Label conditions** as `reward` or `neutral` using robust text matching.
- **Compute a PSC window mean** for each cue and build a tidy trial table (`trial_df`).

At the end you'll have one long-format table with one row per trial per ROI (NAcc + VTA).

In [ ]:
# ════════════════════════════════════════════════════════════════════
# 3. Build a trial-level table from cue events (PSC window mean)
# ════════════════════════════════════════════════════════════════════

# 3a) Load events.tsv
events = pd.read_csv(str(EVENT_PATH), sep='\t')
print("events.columns:", list(events.columns))

unique_types = pd.Series(events["trial_type"].astype(str).unique())
print("first unique trial_type values:")
print(unique_types.head(20).to_list())

# 3b) Get TR from header
TR = image.load_img(str(BOLD_PATH)).header.get_zooms()[3]
print(f"Repetition time (TR) = {TR} s")

# 3c) Cue-only subset (case-insensitive)
trial_type = events["trial_type"].astype(str)
cue_mask = trial_type.str.lower().str.startswith("cue", na=False)
cue_events = events.loc[cue_mask].copy()

# 3d) Condition labels (robust)
trial_type_cue = cue_events["trial_type"].astype(str)
is_reward = trial_type_cue.str.contains("Reward|Gain", case=False, na=False)
is_neutral = trial_type_cue.str.contains("Neutral|Triangle", case=False, na=False)

cue_events = cue_events.loc[is_reward | is_neutral].copy()
cue_events["condition"] = np.where(
    cue_events["trial_type"].astype(str).str.contains("Reward|Gain", case=False, na=False),
    "reward",
    "neutral",
)

# 3e) Trialwise PSC window mean helper
def trial_window_mean(psc_ts, onset_s, TR, lag_trs=5, win_trs=2):
    onset_tr = int(np.round(onset_s / TR))
    start = onset_tr + lag_trs
    end = start + win_trs
    if start < 0 or end > len(psc_ts):
        return np.nan
    return float(np.nanmean(psc_ts[start:end]))

# 3f) Build long-format trial_df (NAcc + VTA)
rows = []
for roi in [nacc, vta]:
    roi_name = roi["roi_name"]
    psc_ts = roi["psc_ts"]
    for _, trial in cue_events.iterrows():
        rows.append(
            {
                "roi": roi_name,
                "condition": trial["condition"],
                "onset_s": float(trial["onset"]),
                "psc_window_mean": trial_window_mean(psc_ts, trial["onset"], TR),
            }
        )

trial_df = pd.DataFrame(rows)

before = len(trial_df)
trial_df = trial_df.dropna(subset=["psc_window_mean"]).reset_index(drop=True)
dropped = before - len(trial_df)
print(f"Dropped {dropped} trial rows with NaNs (out-of-range windows).")

print("trial_df preview:")
print(trial_df.head())

**What this output tells us**:

- The `events.columns` printout confirms the file loaded correctly.
- The first unique `trial_type` values give a quick sanity check of the event labels.
- The printed **TR** tells us how many seconds each volume represents.
- `trial_df` is a tidy, long-format table with one row per trial per ROI (NAcc + VTA) for cue trials only.


### Quick Check: Mean PSC by Condition (NAcc + VTA)

Now that `trial_df` is built, compute mean PSC for both ROIs on reward vs neutral cue trials.

First try writing the code yourself before using AI help.

Fill in code to compute:
- `mean_reward_nacc`, `mean_neutral_nacc`
- `mean_reward_vta`, `mean_neutral_vta`

Then compute and print:
- `delta_nacc = mean_reward_nacc - mean_neutral_nacc`
- `delta_vta  = mean_reward_vta  - mean_neutral_vta`

In 1-2 sentences: do NAcc and VTA show the same direction (sign) for ΔPSC?

In [ ]:
# --- Student TO-DO: compute means for NAcc and VTA (cue trials only) ---
mean_reward_nacc  = ...  # average NAcc PSC for reward cues
mean_neutral_nacc = ...  # average NAcc PSC for neutral cues

mean_reward_vta   = ...  # average VTA PSC for reward cues
mean_neutral_vta  = ...  # average VTA PSC for neutral cues

delta_nacc = mean_reward_nacc - mean_neutral_nacc
delta_vta  = mean_reward_vta  - mean_neutral_vta

print("NAcc delta PSC (Reward - Neutral) =", delta_nacc)
print("VTA  delta PSC (Reward - Neutral) =", delta_vta)

In [ ]:
# --- Student TO-DO: compute means for NAcc and VTA (cue trials only) ---
mean_reward_nacc  = trial_df[(trial_df["roi"] == "NAcc") & (trial_df["condition"] == "reward")]["psc_window_mean"].mean()
mean_neutral_nacc = trial_df[(trial_df["roi"] == "NAcc") & (trial_df["condition"] == "neutral")]["psc_window_mean"].mean()

mean_reward_vta   = trial_df[(trial_df["roi"] == "VTA") & (trial_df["condition"] == "reward")]["psc_window_mean"].mean()
mean_neutral_vta  = trial_df[(trial_df["roi"] == "VTA") & (trial_df["condition"] == "neutral")]["psc_window_mean"].mean()

delta_nacc = mean_reward_nacc - mean_neutral_nacc
delta_vta  = mean_reward_vta  - mean_neutral_vta

print("NAcc delta PSC (Reward - Neutral) =", delta_nacc)
print("VTA  delta PSC (Reward - Neutral) =", delta_vta)

## 4. Visualisations

Below are the key plots you will generate.  Each cell saves its figure into `figs/` so you can easily pull them into your write‑up. The **Figs** folder is in the same folder as this file alongside our datasets.

### 4a) Cue-Locked Timecourses (NAcc + VTA)

We will plot cue-locked PSC timecourses for **NAcc** and **VTA** using the same window and baseline correction.

Each figure shows:
- Reward cue mean PSC vs neutral cue mean PSC
- Shaded SEM

Saved outputs:
- `figs/4a_timecourse_cue_nacc.png`
- `figs/4a_timecourse_cue_vta.png`

In [ ]:
# Cue-locked timecourses for NAcc and VTA
# Required inputs from earlier in the notebook
missing = [name for name in ["nacc", "vta", "cue_events", "TR", "FIG_DIR"] if name not in globals()]
if missing:
    raise NameError(f"Missing required variables: {missing}. Run the earlier data/epoch setup cells first.")

# Window: -2 TR to +14 TR
win_pre, win_post = 2, 14

def extract_epoch(ts, onset_s, TR, pre, post):
    onset_tr = int(np.round(onset_s / TR))
    idx = np.arange(onset_tr - pre, onset_tr + post)
    valid = (idx >= 0) & (idx < len(ts))
    out = np.full(idx.shape, np.nan, dtype=float)
    out[valid] = ts[idx[valid]]
    return out


def baseline_correct(epochs, pre_trs):
    base = np.nanmean(epochs[:, :pre_trs], axis=1, keepdims=True)
    return epochs - base


def plot_cue_timecourse(roi_dict, cue_events_df, TR, fig_path):
    t = np.arange(-win_pre, win_post) * TR
    psc_ts = roi_dict["psc_ts"]

    cue_reward_onsets = cue_events_df[cue_events_df["condition"] == "reward"]["onset"].values
    cue_neutral_onsets = cue_events_df[cue_events_df["condition"] == "neutral"]["onset"].values

    epochs_reward = np.vstack([extract_epoch(psc_ts, o, TR, win_pre, win_post) for o in cue_reward_onsets])
    epochs_neutral = np.vstack([extract_epoch(psc_ts, o, TR, win_pre, win_post) for o in cue_neutral_onsets])

    epochs_reward_bc = baseline_correct(epochs_reward, win_pre)
    epochs_neutral_bc = baseline_correct(epochs_neutral, win_pre)

    m_reward = np.nanmean(epochs_reward_bc, axis=0)
    m_neutral = np.nanmean(epochs_neutral_bc, axis=0)
    se_reward = np.nanstd(epochs_reward_bc, axis=0) / np.sqrt(np.sum(~np.isnan(epochs_reward_bc), axis=0))
    se_neutral = np.nanstd(epochs_neutral_bc, axis=0) / np.sqrt(np.sum(~np.isnan(epochs_neutral_bc), axis=0))

    plt.figure(figsize=(7, 4))
    plt.plot(t, m_neutral, color="C0", label="Neutral cue")
    plt.fill_between(t, m_neutral - se_neutral, m_neutral + se_neutral, color="C0", alpha=0.2)
    plt.plot(t, m_reward, color="C1", label="Reward cue")
    plt.fill_between(t, m_reward - se_reward, m_reward + se_reward, color="C1", alpha=0.2)
    plt.axvline(0, color="k", linestyle=":", linewidth=1)
    plt.xlabel("Time from cue (s)")
    plt.ylabel("PSC (%)")
    plt.title(f"Cue-locked PSC: {roi_dict['roi_name']}")
    plt.legend()
    plt.tight_layout()
    plt.savefig(str(fig_path), dpi=150, bbox_inches="tight")
    plt.show()

plot_cue_timecourse(nacc, cue_events, TR, FIG_DIR / "4a_timecourse_cue_nacc.png")
plot_cue_timecourse(vta, cue_events, TR, FIG_DIR / "4a_timecourse_cue_vta.png")

### Interpreting the Cue-Locked Timecourses

- Check that both ROIs show a cue-locked rise in PSC, with reward higher than neutral.
- The peak should appear around 5 to 7 seconds after cue onset due to hemodynamic lag.
- PSC is BOLD percent-signal change (blood-flow proxy), not dopamine.

### 4b) Box Plots: Cue Window Means (NAcc + VTA)

We will plot Reward vs Neutral cue PSC window means for each ROI.

Saved outputs:
- `figs/4b_box_cue_nacc.png`
- `figs/4b_box_cue_vta.png`

In [ ]:
# ════════════════════════════════════════════════════════════════════
# Box plots for cue window means (NAcc + VTA)
# ════════════════════════════════════════════════════════════════════

def plot_roi_boxplot(roi_name, fig_path):
    roi_trials = trial_df[trial_df["roi"] == roi_name].copy()
    plt.figure(figsize=(5.5, 4))
    sns.boxplot(
        data=roi_trials,
        x="condition",
        y="psc_window_mean",
        order=["neutral", "reward"],
        palette=["C0", "C1"],
        showfliers=False,
    )
    plt.xticks([0, 1], ["Neutral cue", "Reward cue"])
    plt.ylabel("PSC (%)")
    plt.title(f"{roi_name} PSC by cue type")
    plt.tight_layout()
    plt.savefig(str(fig_path), dpi=150, bbox_inches="tight")
    plt.show()

plot_roi_boxplot("NAcc", FIG_DIR / "4b_box_cue_nacc.png")
plot_roi_boxplot("VTA", FIG_DIR / "4b_box_cue_vta.png")

### 4c) Mesolimbic Coupling (Cue Trials)

We will pair VTA and NAcc cue-window PSC per trial and visualize their coupling.

Saved output:
- `figs/4c_vta_nacc_coupling_cue.png`

In [ ]:
# ════════════════════════════════════════════════════════════════════
# Coupling scatter: VTA vs NAcc (cue trials)
# ════════════════════════════════════════════════════════════════════

wide_df = trial_df.pivot_table(
    index=["onset_s", "condition"],
    columns="roi",
    values="psc_window_mean",
    aggfunc="mean",
).reset_index()

wide_df = wide_df.dropna(subset=["NAcc", "VTA"]).reset_index(drop=True)

x = wide_df["VTA"].values
y = wide_df["NAcc"].values

coupling_r = np.nan
if len(x) > 1:
    coupling_r = float(np.corrcoef(x, y)[0, 1])

plt.figure(figsize=(5, 4))
plt.scatter(x, y, alpha=0.7, edgecolor="none")
plt.xlabel("VTA PSC (cue window mean)")
plt.ylabel("NAcc PSC (cue window mean)")
plt.title(f"VTA-NAcc coupling (cue trials), r = {coupling_r:.2f}")
plt.tight_layout()
plt.savefig(str(FIG_DIR / "4c_vta_nacc_coupling_cue.png"), dpi=150, bbox_inches="tight")
plt.show()

### Tiny Summary Table (Cue Trials)

This table summarizes mean PSC by ROI and condition, plus delta PSC (Reward minus Neutral).
Coupling r is reported from the scatterplot as a simple association and does not indicate causal direction.

In [ ]:
# Tiny summary table: ROI x (mean_reward, mean_neutral, delta_psc)
summary = (
    trial_df.groupby(["roi", "condition"])["psc_window_mean"]
           .mean()
           .unstack("condition")
           .rename(columns={"reward": "mean_reward", "neutral": "mean_neutral"})
)

summary["delta_psc"] = summary["mean_reward"] - summary["mean_neutral"]
summary = summary.reset_index()

# Pretty print (rounded)
display(summary.round(4))

# Ensure coupling_r exists (compute from trial_df if needed)
if "coupling_r" not in globals():
    wide_df = trial_df.pivot_table(
        index=["onset_s", "condition"],
        columns="roi",
        values="psc_window_mean",
        aggfunc="mean",
    ).reset_index()
    wide_df = wide_df.dropna(subset=["NAcc", "VTA"]).reset_index(drop=True)

    x = wide_df["VTA"].values
    y = wide_df["NAcc"].values

    coupling_r = np.nan
    if len(x) > 1:
        coupling_r = float(np.corrcoef(x, y)[0, 1])

print("Coupling r (VTA vs NAcc, cue trials) =", round(coupling_r, 3))

## Complete Individual Preparation Slides for Exercise 8